# Simulating Constitutive Processes of semantic change within heterogeneous populations of speakers

In [ ]:
from mod.agent import *
from mod.network import *

In [ ]:
no_agents = 50
no_connections = 10